In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-05-05


Found rundate 2023-05-04 10:00:02 < 2023-05-05 at commit 9c369882; returning commit 2cae8fd


'2023-05-05 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-05-05 10:00:02, NJSP was reporting 163 YTD deaths
Current YTD Deaths (2024-05-05 10:00:03-04:00): 207
Previous year YTD Deaths (adjusted; 2023-05-05 10:00:02): 163.86163171104602
Projected 2024 total: 690.101155799655
26.3% change


FAUQStats cache miss: 2023, Fri May 05 10:00:02 EDT 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

CCODE       CNAME MCODE            MNAME HIGHWAY  \
ACCID                                                    
12249    07       Essex  0714      Newark City     NaN   
12251    01    Atlantic  0112    Hamilton Twsp     322   
12253    12   Middlesex  1217  Piscataway Twsp     NaN   
12254    02      Bergen  0221    Garfield City     NaN   
12267    07       Essex  0714      Newark City     NaN   
...     ...         ...   ...              ...     ...   
12431    16     Passaic  1614       Wayne Twsp      80   
12432    20       Union  2004   Elizabeth City      95   
12434    12   Middlesex  1225  Woodbridge Twsp     440   
12433    08  Gloucester  0806   Glassboro Boro     322   
12437    02      Bergen  0242     Oakland Boro     287   

                          LOCATION  FATALITIES  FATAL_D  FATAL_P  FATAL_T  \
ACCID                                                                       
12249                  Mulberry St         1.0      0.0      0.0      1.0   
12251    State Highway 322 MP 46.4         1.0      1.0      0.0      0.0   
12253       South Randolphville Rd         1.0      1.0      0.0      0.0   
12254                  Outwater Ln         1.0      0.0      0.0      1.0   
12267                Washington St         1.0      0.0      0.0      1.0   
...                            ...         ...      ...      ...      ...   
12431        Interstate 80 MP 53.6         2.0      1.0      1.0      0.0   
12432  New Jersey Turnpike MP 99.9         1.0      0.0      1.0      0.0   
12434     State Highway 440 MP 1.4         1.0      1.0      0.0      0.0   
12433            State Highway 322         1.0      1.0      0.0      0.0   
12437       Interstate 287 MP 59.8         1.0      1.0      0.0      0.0   

       FATAL_B  INJURIES                  STREET                        dt  
ACCID                                                                       
12249      0.0       NaN             Mulberry St 2023-01-01 01:49:00-05:00  
12251      0.0       0.0                     NaN 2023-01-02 09:26:00-05:00  
12253      0.0       0.0  South Randolphville Rd 2023-01-02 19:41:00-05:00  
12254      0.0       0.0             Outwater Ln 2023-01-03 20:52:00-05:00  
12267      0.0       NaN           Washington St 2023-01-04 06:21:00-05:00  
...        ...       ...                     ...                       ...  
12431      0.0       2.0                     NaN 2023-04-30 10:04:00-04:00  
12432      0.0       NaN                     NaN 2023-04-30 22:39:00-04:00  
12434      0.0       NaN                     NaN 2023-05-01 12:24:00-04:00  
12433      0.0       NaN                     NaN 2023-05-01 16:06:00-04:00  
12437      0.0       0.0                     NaN 2023-05-04 07:48:00-04:00  

[157 rows x 14 columns]

Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

driver  passenger  pedestrian  cyclist  crashes
county                                                     
Atlantic         7          1           5        1       14
Bergen           7          2           9        0       18
Burlington      11          3           2        0       16
Camden           6          2           9        0       17
Cape May         1          0           0        0        1
Cumberland       5          1           1        0        7
Essex            5          3           5        0       13
Gloucester       5          2           0        0        7
Hudson           2          2           1        0        4
Hunterdon        0          0           2        0        2
Mercer           6          4           3        0       12
Middlesex       12          4           9        0       21
Monmouth        12          3           6        1       21
Morris           2          0           2        0        4
Ocean            7          1           6        0       14
Passaic          8          0           1        0        9
Salem            3          1           1        0        4
Somerset         1          0           2        0        3
Sussex           1          0           1        0        2
Union            2          0           4        0        6
Warren           2          0           1        1        4

Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

34.3% through the year, 65.7% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

prv_ytd  prv_end  cur_ytd
county     type                                 
Atlantic   driver            5       17        7
Bergen     driver            5       20        7
Burlington driver            3       26       11
Camden     driver            5       18        6
Cape May   driver            2        4        1
...                        ...      ...      ...
Sussex     driver            0        6        1
           passenger         0        2        0
           pedestrian        0        1        1
           cyclist           0        0        0
           crashes           0        6        2

[105 rows x 3 columns]

In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type        crashes  cyclist  driver  passenger  pedestrian
county                                                     
Atlantic         41        3      21          7          13
Bergen           45        1      24          6          14
Burlington       60        1      55          7           4
Camden           61        3      20          7          46
Cape May          5        0       3          1           1
Cumberland       22        0      17          3           3
Essex            50        1      18          7          24
Gloucester       30        1      23          6           2
Hudson           18        1      10          5           5
Hunterdon         6        0       2          0           3
Mercer           41        0      18          9          14
Middlesex        55        1      28         20          20
Monmouth         63        4      55         13          10
Morris           17        0      10          3           3
Ocean            48        1      27          7          16
Passaic          32        0      25          0           7
Salem            13        0       8          3           3
Somerset         21        0      10          3           9
Sussex            7        0       6          1           2
Union            22        1       7          2          13
Warren           16        2      10          1           3

In [9]:
projected.to_csv(PROJECTED_CSV)